# Single Graph Analysis
### *(Individual Metrics)*

#### Load packages

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#### Load data

In [2]:
# This is a sampling for entire analysis
data_astronomy1 = pd.read_excel('data/Astronomy.xlsx', sheet_name='강민철')
data_astronomy2 = pd.read_excel('data/Astronomy.xlsx', sheet_name='강지헌')

# Drop an useless column
data_astronomy1.drop('Name', axis=1, inplace=True)
data_astronomy2.drop('Name', axis=1, inplace=True)

# Merge all dataframes into one dataframe:
data_astronomy = pd.concat([data_astronomy1, data_astronomy2], axis=0)
data_astronomy.drop(data_astronomy.loc[data_astronomy['ID'] == 53].index[0], inplace=True)
# data_astronomy

In [3]:
data_sampling1 = pd.read_excel('data/Sampling.xlsx', sheet_name='강지헌')
data_sampling2 = pd.read_excel('data/Sampling.xlsx', sheet_name='신아현')
data_sampling3 = pd.read_excel('data/Sampling.xlsx', sheet_name='신수연')

data_sampling1.drop('Name', axis=1, inplace=True)
data_sampling2.drop('Name', axis=1, inplace=True)
data_sampling3.drop('Name', axis=1, inplace=True)

data_sampling = pd.concat([data_sampling1, data_sampling2, data_sampling3], axis=0)
# data_sampling

In [4]:
data_database1 = pd.read_excel('data/Database.xlsx', sheet_name='신수연')
data_database2 = pd.read_excel('data/Database.xlsx', sheet_name='양연선')
data_database3 = pd.read_excel('data/Database.xlsx', sheet_name='김나영')

data_database1.drop('Name', axis=1, inplace=True)
data_database2.drop('Name', axis=1, inplace=True)
data_database3.drop('Name', axis=1, inplace=True)

data_database = pd.concat([data_database1, data_database2, data_database3], axis=0)
# data_database

#### Process graphs
- 이름 형식: `<Domain>_<Modality>_<ID>`
- Domain: `ASTRONOMY`, `SAMPLING`, `DATABASE`

***참고사항**:
    혹시 몰라 노드에 P.Knowledge를 추가로 라벨링해두었지만, 분석할 때는 그냥 df상에서도 충분히 가능해보임*

In [5]:
graphs_astronomy = {}

for id, sub_df in data_astronomy.groupby('ID'):
    # New graph object
    graph_name = f"Astronomy_{sub_df['Mod.'].iloc[0]}_{sub_df['ID'].iloc[0]}"
    G = nx.DiGraph()
    
    # Add nodes and edges
    for _, row in sub_df.iterrows():
        start_node = row['Start']
        if pd.notna(row['End']):
            end_nodes = [end_node.rstrip() for end_node in row['End'].split(',')]
            for end_node in end_nodes:
                G.add_edge(start_node, end_node)
        # Add p.knowledge labels:  O -> 1(true)  |  X -> 0(false)
        try:
            G.nodes[start_node]['P.Knowledge'] = 1 if row['P.Knowledge'] == 'O' else 0
        except KeyError:
            G.add_node(start_node)
            G.nodes[start_node]['P.Knowledge'] = 0
    
    # Save the graph
    graphs_astronomy[graph_name] = G

In [6]:
graphs_sampling = {}

for id, sub_df in data_sampling.groupby('ID'):
    # New graph object
    graph_name = f"Sampling_{sub_df['Mod.'].iloc[0]}_{sub_df['ID'].iloc[0]}"
    G = nx.DiGraph()
    
    # Add nodes and edges
    for _, row in sub_df.iterrows():
        start_node = row['Start']
        if pd.notna(row['End']):
            end_nodes = [end_node.rstrip() for end_node in row['End'].split(',')]
            for end_node in end_nodes:
                G.add_edge(start_node, end_node)
        # Add p.knowledge labels:  O -> 1(true)  |  X -> 0(false)
        try:
            G.nodes[start_node]['P.Knowledge'] = 1 if row['P.Knowledge'] == 'O' else 0
        except KeyError:
            G.add_node(start_node)
            G.nodes[start_node]['P.Knowledge'] = 0
    
    # Save the graph
    graphs_sampling[graph_name] = G

In [7]:
graphs_database = {}

for id, sub_df in data_database.groupby('ID'):
    # New graph object
    graph_name = f"Database_{sub_df['Mod.'].iloc[0]}_{sub_df['ID'].iloc[0]}"
    G = nx.DiGraph()
    
    # Add nodes and edges
    for _, row in sub_df.iterrows():
        start_node = row['Start']
        if pd.notna(row['End']):
            end_nodes = [end_node.rstrip() for end_node in row['End'].split(',')]
            for end_node in end_nodes:
                G.add_edge(start_node, end_node)
        # Add p.knowledge labels:  O -> 1(true)  |  X -> 0(false)
        try:
            G.nodes[start_node]['P.Knowledge'] = 1 if row['P.Knowledge'] == 'O' else 0
        except KeyError:
            G.add_node(start_node)
            G.nodes[start_node]['P.Knowledge'] = 0
    
    # Save the graph
    graphs_database[graph_name] = G

#### Get roots

In [8]:
# Conversion of the dataframes into dictionaries
def get_root_dict(df: pd.DataFrame) -> dict:
    result_dict = {}
    for id, sub_df in df.groupby('ID'):
        result_dict[str(id)] = sub_df[sub_df['Root'] == 'O']['Start'].tolist()
    return result_dict

In [9]:
roots_astronomy = get_root_dict(data_astronomy)
roots_sampling = get_root_dict(data_sampling)
roots_database = get_root_dict(data_database)

# If you want to see the dictionaries, you can uncomment the following lines
# print(roots_astronomy)
# print(roots_sampling)
# print(roots_database)

In [10]:
import re

def extract_id(text):
    pattern = r'\d+$'
    match = re.search(pattern, text)
    if match:
        return match.group()
    return None

#### Methods for individual metrics
1. `num_of_concepts`:
   - 노드 개수와 동일
2. `num_of_relationships`
   - 엣지 개수와 동일
3. `num_of_hierarchies`:
   - 제대로 이해한 것이 맞다면, deepest hierarchy와 1 차이나는 별 의미 없는 값
4. `num_of_crosslinks`:
   - MST를 생성하여 MST에 포함되지 않은 엣지 개수를 카운트
5. `cal_aspl`:
   - 가중평균 값 계산
6. `cal_cc`:
   - 가중평균 값 계산
   - 삼각형 구조가 없으면 0이 될 수 있음
7. `cal_network_density`
8. `cal_complexity`
9.  `get_level_deepest_hierarchy`:
    - Root로부터 가장 긴 체인의 길이 (Root 포함)
    - 2개 이상의 root 존재 시, max 값 선택
10. `num_of_components`

In [11]:
def num_of_concepts(G):
    return G.number_of_nodes()

def num_of_relationships(G):
    return G.number_of_edges()

# def num_of_hierarchies(G):
#     return nx.number_of_strongly_connected_components(G)

def num_of_crosslinks(G):
    crosslink_count = 0
    
    # Handle connected components separately
    if G.is_directed():
        components = [G.subgraph(c).copy() for c in nx.weakly_connected_components(G)]
    else:
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    
    for component in components:
        # Convert directed graph to undirected for minimum spanning tree calculation
        undirected_component = component.to_undirected()
        # Calculate the minimum spanning tree (MST) of the undirected component
        mst = nx.minimum_spanning_tree(undirected_component)
        
        # Crosslinks are edges in the original component that are not in the MST
        for u, v in component.edges():
            if not mst.has_edge(u, v):
                crosslink_count += 1
    
    return crosslink_count

def cal_aspl(G):
    # 강제 변환
    if G.is_directed():
        G = G.to_undirected()

    # Find components
    components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    
    total_aspl = 0
    total_nodes = 0
    
    for component in components:
        n = len(component)
        if n > 1:  # 최소 두 개의 노드가 있는 컴포넌트만 고려
            try:
                aspl = nx.average_shortest_path_length(component)
                total_aspl += aspl * n
                total_nodes += n
            except nx.NetworkXError:
                continue
    
    # 가중 평균
    avg_aspl = total_aspl / total_nodes if total_nodes > 0 else 0

    return avg_aspl

def cal_cc(G):
    # Find components
    components = [G.subgraph(c).copy() for c in nx.weakly_connected_components(G)]
    
    # Initialize total sum and total nodes count
    total_weighted_cc = 0
    total_nodes = 0
    
    # Calculate weighted CC for each component
    for component in components:
        num_nodes = component.number_of_nodes()
        component_cc = nx.average_clustering(component)
        total_weighted_cc += component_cc * num_nodes
        total_nodes += num_nodes
    
    # Calculate weighted average CC
    avg_cc = total_weighted_cc / total_nodes if total_nodes > 0 else 0
    
    return avg_cc

def cal_network_density(G):
    return nx.density(G)

def cal_complexity(G):
    E = G.number_of_edges()
    N = G.number_of_nodes()
    return (E / N) if (N > 0) else 0

def get_level_deepest_hierarchy(G, roots_list):
    max_hierarchy_level = 0
    
    # Handle connected components separately
    if G.is_directed():
        components = [G.subgraph(c).copy() for c in nx.weakly_connected_components(G)]
    else:
        components = [G.subgraph(c).copy() for c in nx.connected_components(G)]
    
    # Iterate through each component and find the maximum hierarchy level
    for component in components:
        for root in roots_list:
            if root in component:  # root가 component에 있는지 확인
                # Calculate the depth of the hierarchy from this root node
                hierarchy_levels = nx.single_source_shortest_path_length(component, root).values()
                max_hierarchy_level = max(max_hierarchy_level, max(hierarchy_levels))
    
    return max_hierarchy_level + 1  # Adding 1 to include the root level itself

def num_of_components(G):
    if G.is_directed():
        return len(list(nx.weakly_connected_components(G)))
    else:
        return len(list(nx.connected_components(G)))

#### Analysis for individual metrics

In [12]:
# Set a new dataframe for the analysis
columns = ["ID", "num_of_concepts", "num_of_relationships", "num_of_crosslinks", "ASPL", "CC", "network_density", "complexity", "level_of_deepest_hierarchy", "num_of_components"]
indiv_astronomy = pd.DataFrame(columns=columns).set_index("ID")
indiv_sampling = pd.DataFrame(columns=columns).set_index("ID")
indiv_database = pd.DataFrame(columns=columns).set_index("ID")

In [13]:
for graph_name, G in graphs_astronomy.items():
    try:
        indiv_astronomy.loc[graph_name] = [
            num_of_concepts(G), 
            num_of_relationships(G), 
            num_of_crosslinks(G), 
            cal_aspl(G), 
            cal_cc(G), 
            cal_network_density(G), 
            cal_complexity(G), 
            get_level_deepest_hierarchy(G, roots_astronomy[extract_id(graph_name)]),
            num_of_components(G)
        ]
    except nx.NetworkXNoPath as e:
        print(f"Path error at {graph_name}: {e}")

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
indiv_astronomy.to_excel("result/Astronomy_indiv.xlsx", index=True)

In [14]:
for graph_name, G in graphs_sampling.items():
    try:
        indiv_sampling.loc[graph_name] = [
            num_of_concepts(G), 
            num_of_relationships(G), 
            num_of_crosslinks(G), 
            cal_aspl(G), 
            cal_cc(G), 
            cal_network_density(G), 
            cal_complexity(G), 
            get_level_deepest_hierarchy(G, roots_sampling[extract_id(graph_name)]),
            num_of_components(G)
        ]
    except nx.NetworkXNoPath as e:
        print(f"Path error at {graph_name}: {e}")

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
indiv_sampling.to_excel("result/Sampling_indiv.xlsx", index=True)

In [15]:
for graph_name, G in graphs_database.items():
    try:
        indiv_database.loc[graph_name] = [
            num_of_concepts(G), 
            num_of_relationships(G), 
            num_of_crosslinks(G), 
            cal_aspl(G), 
            cal_cc(G), 
            cal_network_density(G), 
            cal_complexity(G), 
            get_level_deepest_hierarchy(G, roots_database[extract_id(graph_name)]),
            num_of_components(G)
        ]
    except nx.NetworkXNoPath as e:
        print(f"Path error at {graph_name}: {e}")

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
indiv_database.to_excel("result/Database_indiv.xlsx", index=True)

#### Methods for centrality metrics

1. `cal_node_betweeness`
2. `cal_node_closeness`
3. `cal_node_degree`

*(각각 상위 3개의 항목을 추출함)*

In [16]:
def cal_node_betweenness(G):
    values = nx.betweenness_centrality(G)
    top_3_nodes = sorted(values.items(), key=lambda item: item[1], reverse=True)[:3]
    return dict(top_3_nodes)

def cal_node_closeness(G):
    values = nx.closeness_centrality(G)
    top_3_nodes = sorted(values.items(), key=lambda item: item[1], reverse=True)[:3]
    return dict(top_3_nodes)

def cal_node_degree(G):
    values = nx.degree_centrality(G)
    top_3_nodes = sorted(values.items(), key=lambda item: item[1], reverse=True)[:3]
    return dict(top_3_nodes)

def cal_betweenness_average(G):
    G = nx.DiGraph(G)
    values = nx.betweenness_centrality(G)
    return np.mean(list(values.values()))

def cal_closeness_average(G):
    G = nx.DiGraph(G)
    values = nx.closeness_centrality(G)
    return np.mean(list(values.values()))

def cal_degree_average(G):
    G = nx.DiGraph(G)
    values = nx.degree_centrality(G)
    return np.mean(list(values.values()))

#### Analysis for centrality metrics

In [17]:
def add_centrality_to_dataframe(df, graph_name, centrality_data, centrality_type, mean_value):
    df.loc[graph_name, (centrality_type, 'node1')] = list(centrality_data.keys())[0]
    df.loc[graph_name, (centrality_type, 'value1')] = list(centrality_data.values())[0]
    df.loc[graph_name, (centrality_type, 'node2')] = list(centrality_data.keys())[1]
    df.loc[graph_name, (centrality_type, 'value2')] = list(centrality_data.values())[1]
    df.loc[graph_name, (centrality_type, 'node3')] = list(centrality_data.keys())[2]
    df.loc[graph_name, (centrality_type, 'value3')] = list(centrality_data.values())[2]
    
    df.loc[graph_name, (centrality_type, 'mean')] = mean_value

In [18]:
# Get the most important node in each graph
columns = pd.MultiIndex.from_product([['Betweenness', 'Closeness', 'Degree'], ['node1', 'value1', 'node2', 'value2', 'node3', 'value3', 'mean']])
central_astronomy = pd.DataFrame(columns=columns)
central_sampling = pd.DataFrame(columns=columns)
central_database = pd.DataFrame(columns=columns)

In [19]:
for graph_name, G in graphs_astronomy.items():
    betweenness_top_3 = cal_node_betweenness(G)
    closeness_top_3 = cal_node_closeness(G)
    degree_top_3 = cal_node_degree(G)
    
    add_centrality_to_dataframe(central_astronomy, graph_name, betweenness_top_3, 'Betweenness', cal_betweenness_average(G))
    add_centrality_to_dataframe(central_astronomy, graph_name, closeness_top_3, 'Closeness', cal_closeness_average(G))
    add_centrality_to_dataframe(central_astronomy, graph_name, degree_top_3, 'Degree', cal_degree_average(G))

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
central_astronomy.to_excel("result/Astronomy_central.xlsx", index=True)

In [20]:
for graph_name, G in graphs_sampling.items():
    betweenness_top_3 = cal_node_betweenness(G)
    closeness_top_3 = cal_node_closeness(G)
    degree_top_3 = cal_node_degree(G)
    
    add_centrality_to_dataframe(central_sampling, graph_name, betweenness_top_3, 'Betweenness', cal_betweenness_average(G))
    add_centrality_to_dataframe(central_sampling, graph_name, closeness_top_3, 'Closeness', cal_closeness_average(G))
    add_centrality_to_dataframe(central_sampling, graph_name, degree_top_3, 'Degree', cal_degree_average(G))

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
central_sampling.to_excel("result/Sampling_central.xlsx", index=True)

In [21]:
for graph_name, G in graphs_database.items():
    betweenness_top_3 = cal_node_betweenness(G)
    closeness_top_3 = cal_node_closeness(G)
    degree_top_3 = cal_node_degree(G)
    
    add_centrality_to_dataframe(central_database, graph_name, betweenness_top_3, 'Betweenness', cal_betweenness_average(G))
    add_centrality_to_dataframe(central_database, graph_name, closeness_top_3, 'Closeness', cal_closeness_average(G))
    add_centrality_to_dataframe(central_database, graph_name, degree_top_3, 'Degree', cal_degree_average(G))

# Save the result to excel file
# ❗WARNING❗ It will automatically overwrite the existing file!!!
central_database.to_excel("result/Database_central.xlsx", index=True)